In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import KNNImputer

from geopy.distance import geodesic

import plotly.express as px

import holidays

import scipy.stats as stats


In [ ]:
# Carga el dataset en un dataframe
df = pd.read_csv('uber_fares.csv')

# Revisa si hay filas duplicadas
df.duplicated().sum() # 0 filas duplicadas

# Revisa las columnas y sus tipos de datos
df.dtypes


In [ ]:
# Asigna el tipo de datos correcto a las variables que representan fechas
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime']).dt.floor('s')
df['date'] = pd.to_datetime(df['date']).dt.floor('s')

# Muestra las primeras filas del dataframe
df.head()

# Limpieza y preprocesamiento

In [ ]:
# Elimina la columna 'key' que no aporta información relevante
df = df.drop(columns=['key'])

# Chequea si las columnas 'pickup_datetime' y 'date' son iguales
df['pickup_datetime'].equals(df['date']) # True

# Elimina la columna 'pickup_datetime' ya que es redundante
df = df.drop(columns=['pickup_datetime'])

In [ ]:
# Elimina las filas cuya variable objetivo no es un valor posible
df = df[df['fare_amount'] > 0] # 0.01% de los datos

#### Null Island
Isla ficticia, ubicada en 0°N 0°E, que los GPS suelen utilizar como ubicación por defecto cuando fallan y no pueden determinar la ubicación real, es decir, representa una ubicación nula.

In [ ]:
df_sin_null_island = df[
    ~(
        (df['pickup_latitude'] == 0) & (df['pickup_longitude'] == 0) |
        (df['dropoff_latitude'] == 0) & (df['dropoff_longitude'] == 0)
    )
]

print(f"Los viajes que comenzaron o terminaron en Null Island representan el {100 * (1 - df_sin_null_island.shape[0] / df.shape[0]):.2f}% de los datos.")

In [ ]:
# Elimina las filas que corresponden a viajes que empezaron o terminaron en Null Island
df = df_sin_null_island

### Visualización de las variables de coordenadas

In [ ]:
# Genera un nuevo dataframe con todas las coordenadas
coordenadas = pd.concat(
    [
        df[['pickup_latitude', 'pickup_longitude']].rename(
            columns={'pickup_latitude': 'latitude', 'pickup_longitude': 'longitude'}
        ).assign(type='pickup'),
        df[['dropoff_latitude', 'dropoff_longitude']].rename(
            columns={'dropoff_latitude': 'latitude', 'dropoff_longitude': 'longitude'}
        ).assign(type='dropoff')
    ], ignore_index=True
)

# Muestra un mapa de las ubicaciones de inicio y fin del viaje
fig = px.scatter_map(
    coordenadas.sample(50000),
    lat="latitude",
    lon="longitude",
    color="type",
    zoom=5,
    title="Ubicaciones"
)

fig.update_layout(
    mapbox_style="open-street-map",
    margin={"r":0,"t":30,"l":0,"b":0}  # elimina márgenes blancos
)
fig.show()


A simple vista se observa que hay una gran densidad de viajes en Nueva York

In [ ]:
limites_ny = {
    'lat_min': 40.49,
    'lat_max': 40.92,
    'lon_min': -74.27,
    'lon_max': -73.68
}

cantidad_viajes = df.shape[0]

# Filtra los viajes que se dieron de los límites de Nueva York
df_ny = df[
    (df['pickup_latitude'] >= limites_ny['lat_min']) &
    (df['pickup_latitude'] <= limites_ny['lat_max']) &
    (df['pickup_longitude'] >= limites_ny['lon_min']) &
    (df['pickup_longitude'] <= limites_ny['lon_max']) &
    (df['dropoff_latitude'] >= limites_ny['lat_min']) &
    (df['dropoff_latitude'] <= limites_ny['lat_max']) &
    (df['dropoff_longitude'] >= limites_ny['lon_min']) &
    (df['dropoff_longitude'] <= limites_ny['lon_max'])
]
cantidad_viajes_ny = df_ny.shape[0]

print(f"Los viajes dentro de los límites de Nueva York representan el {cantidad_viajes_ny / cantidad_viajes * 100:.2f}% del total.")


TODO Vamos a reducir el alcance de nuestro modelo predictivo a viajes integramente dentro de la ciudad de Nueva York, ya que etc COMPLETAR

In [ ]:
# Elimina las filas que corresponden a viajes que no se dieron dentro de los límites de Nueva York
df = df_ny

Los valores posibles para la variable *passenger_count* están en el rango [0;6]. (UberX permite hasta 4 y UberXL hasta 6).

In [ ]:
# Análisis de viajes con cantidad de pasajeros no válida
df[(df['passenger_count'] > 6)]

In [ ]:
# Elimina la fila con passenger_count = 208, que es la única con valor absurdo.
df = df[df['passenger_count'] <= 6]

In [ ]:
print(f"Hasta este momento eliminamos un {100 - df.shape[0] / 200000 * 100:.2f}% de los datos.")

### Generación de variable distancia

In [ ]:
def imputar_distancia(viaje):
    '''
    Calcula la distancía  en kilometros del viaje
    mediante una combinación de distancia Manhattan con
    distancia geodésica (teniendo en cuenta la curvatura
    de la Tierra.)
    '''
    lat1 = viaje['pickup_latitude']
    lon1 = viaje['pickup_longitude']
    lat2 = viaje['dropoff_latitude']
    lon2 = viaje['dropoff_longitude']

    distancia_lat = np.float32(geodesic((lat1, lon1), (lat2, lon1)).kilometers)
    distancia_lon = np.float32(geodesic((lat2, lon1), (lat2, lon2)).kilometers)

    return distancia_lat + distancia_lon

df['distance'] = df.apply(imputar_distancia, axis=1)

### Split Train/Test

In [ ]:
# Separa el 80% para train y 20% para test
train, test= train_test_split(df, test_size=0.2, random_state=1)

## EDA

In [ ]:
# Distribución de variables
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

sns.histplot(data=train, x='fare_amount', bins=50, color=sns.color_palette("muted")[0], ax=axes[0], edgecolor='none')
sns.histplot(data=train, x='distance', bins=50, color=sns.color_palette("muted")[1], ax=axes[1], edgecolor='none')
sns.countplot(data=train, x='passenger_count', color=sns.color_palette("muted")[2], ax=axes[2])

plt.tight_layout()
plt.show()

Se observa que tanto *fare_amount* como *distance* están sesgadas hacía la derecha. Vamos a tratar sus outliers.

### Tratado de Outliers

#### Variable *fare_amount*

In [ ]:
train['fare_amount'].describe(percentiles=[0.001, 0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99, 0.999])

In [ ]:
# Análisis de tarifas mayores al 99.9% de los datos.
sns.histplot(data=train[train['fare_amount'] > 75], x='fare_amount', bins=50, color=sns.color_palette("muted")[0])

Se obversan 4 valores atípicos de tarifa que se separan mucho del resto de los datos y también entre si. Los reemplazamos por valores nulos para su posterior imputación.

In [ ]:
train['fare_amount'] = np.where(train['fare_amount'] > 150, np.nan, train['fare_amount'])
test['fare_amount'] = np.where(test['fare_amount'] > 150, np.nan, test['fare_amount'])

In [ ]:
# Análisis de las tarifas hasta el percentil 1%
train[train['fare_amount'] <= 3.3]['fare_amount'].value_counts()

Se observan 2 valores de tarifa muy inferiores en valor y frecuencia a los demás datos. Las reemplazamos por nulos para para su posterior imputación

In [ ]:
train['fare_amount'] = np.where(train['fare_amount'] < 1, np.nan, train['fare_amount'])
test['fare_amount'] = np.where(test['fare_amount'] < 1, np.nan, test['fare_amount'])

#### Variable *distance*

In [ ]:
train['distance'].describe()

La distancia del viaje no puede ser 0, porque no habría viaje. Asumimos que corresponden a viajes que comienzan y terminan en la misma ubicación pero tienen un recorrido con paradas intermedias, como no tenemos información sobre paradas intermedias, reemplazamos los valores 0 de distancia por nulos para su posterior imputacíon.

In [ ]:
train = train.assign(distance=lambda x: x['distance'].replace(0, np.nan))
test = test.assign(distance=lambda x: x['distance'].replace(0, np.nan))

# Estadísticas descriptivas de distancias no nulas
train['distance'].describe(percentiles=[0.001, 0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99, 0.999])

No vamos a considerar como válida ninguna distancia menor a 0.1km. Reemplazamos por null para posterior imputación.

In [ ]:
train['distance'] = np.where(train['distance'] < 0.1, np.nan, train['distance'])
test['distance'] = np.where(test['distance'] < 0.1, np.nan, test['distance'])

In [ ]:
# Descarta viajes sin tarifa y sin distancia. No imputables.
train = train[~(train['fare_amount'].isnull() & train['distance'].isnull())]

### Escalado

In [ ]:
scaler = StandardScaler()
scaler.fit(train[['distance', 'fare_amount']])

train[['distance', 'fare_amount']] = scaler.transform(train[['distance', 'fare_amount']])
test[['distance', 'fare_amount']] = scaler.transform(test[['distance', 'fare_amount']])

### Imputación

In [ ]:
# Imputa los valores faltantes de las variables numéricas fare_amount y distance mediante KNNImputer
imputer = KNNImputer(n_neighbors=5)
imputer.fit(train[['distance', 'fare_amount']])

train[['distance', 'fare_amount']] = imputer.transform(train[['distance', 'fare_amount']])
test[['distance', 'fare_amount']] = imputer.transform(test[['distance', 'fare_amount']])


In [ ]:
plt.figure(figsize=(12, 6))

sns.scatterplot(
    data=train,
    x='distance',
    y='fare_amount',
    color=sns.color_palette("muted")[0],
)

plt.xlim(-1, 5)
plt.ylim(-1, 5)
plt.xlabel("Distancia (escalada)")
plt.ylabel("Tarifa (escalada)")
plt.title("Relación entre distancia y tarifa (escaladas)")
plt.show()

### Ingenieria de características

#### Variable *date*

In [ ]:
train['hour'] = train['date'].dt.hour
train['day'] = train['date'].dt.day_of_week
train['month'] = train['date'].dt.month

test['hour'] = test['date'].dt.hour
test['day'] = test['date'].dt.day_of_week
test['month'] = test['date'].dt.month

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(18, 12))

dias_semana = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]

for i, dia in enumerate(dias_semana):
    ax = axes[i // 3, i % 3]
    sns.histplot(
        data=train[train["day"] == i],
        x="hour",
        bins=24,
        color=sns.color_palette("muted", 7)[i],
        edgecolor="none",
        ax=ax
    )
    ax.set_title(dia)
    ax.set_xlabel("Horas")
    ax.set_ylabel("Frecuencia")
    ax.set_xticks(range(0, 24, 2))

fig.delaxes(axes[2, 1])
fig.delaxes(axes[2, 2])

plt.tight_layout()
plt.show()

Se observa que los días Lunes, Martes y Miércoles presentan una distribución muy parecida, con una rápida diminución de viajes luego de las 22 horas.


Los Jueves, Viernes y Sábados presentan una mayor cantidad de actividad en horarios nocturnos, y un aumento en la demanda luego de las 22 horas, especialmente los viernes y sábados.


Los domingos presentan un comportamiento único, teníendo su momento de mayor actividad a la madrugada y una baja de actividad a partir de las 18.

In [ ]:
# Genera variables dummys de grupos de dias de la semana con distribuciones similares
train['is_mon_tue_wed'] = np.where(train['day'].isin([0, 1, 2]), 1, 0)
train['is_thu_fri_sat'] = np.where(train['day'].isin([3, 4, 5]), 1, 0)
train['is_sun'] = np.where(train['day'] == 6, 1, 0)

test['is_mon_tue_wed'] = np.where(test['day'].isin([0, 1, 2]), 1, 0)
test['is_thu_fri_sat'] = np.where(test['day'].isin([3, 4, 5]), 1, 0)
test['is_sun'] = np.where(test['day'] == 6, 1, 0)

In [ ]:
tarifa_promedio_por_hora = train.groupby("hour")['fare_amount'].mean().reset_index(name="promedio")

plt.figure(figsize=(12, 6))
sns.barplot(
    data=tarifa_promedio_por_hora,
    x="hour",
    y="promedio",
    hue="hour",
    palette="muted",
    legend=False
)

plt.title("Tarifa promedio por hora del día")
plt.xlabel("Hora")
plt.ylabel("Tarifa Promedio")

plt.tight_layout()
plt.show()

Las tárifas promedio mas altas se dan a las 4 y 5 de la madrugada de forma acentuada por sobre el resto de las horas del día.

In [ ]:
# Genera variable dummy para indicar horario de madrugada
train['is_early_morning'] = np.where(train['hour'].isin([4, 5]), 1, 0)

test['is_early_morning'] = np.where(test['hour'].isin([4, 5]), 1, 0)

In [ ]:
viajes_por_hora = train.groupby("hour")['fare_amount'].size().reset_index(name="frecuencia")

plt.figure(figsize=(12, 6))
sns.barplot(
    data=viajes_por_hora,
    x="hour",
    y="frecuencia",
    hue="hour",
    palette="muted",
    legend=False
)

plt.title("Cantidad de viajes por hora del día")
plt.xlabel("Hora")
plt.ylabel("Frecuencia")

# Mostrar el gráfico
plt.tight_layout()
plt.show()

In [ ]:
train['hour'].value_counts().head(10)

El horario de mayor actividad general se da marcadamente entre las 18 y las 22.

In [ ]:
# Genera variable dummy de alta actividad
train['is_high_activity'] = np.where(train['hour'].isin([18, 19, 20, 21, 22]), 1, 0)

test['is_high_activity'] = np.where(test['hour'].isin([18, 19, 20, 21, 22]), 1, 0)

In [ ]:
tarifa_promedio_por_mes = train.groupby("month")['fare_amount'].mean().reset_index(name="promedio")

plt.figure(figsize=(12, 6))
sns.barplot(
    data=tarifa_promedio_por_mes,
    x="month",
    y="promedio",
    hue="month",
    palette="muted",
    legend=False,
)

plt.xticks(
    ticks=range(12),
    labels=["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"],
    rotation=45
)

plt.title("Tarifa promedio por mes del año")
plt.xlabel("Mes")
plt.ylabel("Tarifa Promedio")

plt.tight_layout()
plt.show()

La tarifa promedio a lo largo de los meses no muestra ninguna tendencia o particularidad clara por lo que decidimos no incluir ninguna variable relacionada para reducir el riesgo de overfitting.

In [ ]:
# Crea variable dummy para identificar viajes en días feriados
us_holidays = holidays.US(years=train["date"].dt.year.unique(), state="NY")
train["is_holiday"] = train["date"].dt.date.isin(us_holidays)

In [ ]:
train.groupby("is_holiday")['fare_amount'].describe()

El hecho de que sea o no feriado no parece afectar el valor de la tarifa. por lo que decidimos eliminarla para reducir el riesgo de overfitting.

In [ ]:
train.drop(columns=['is_holiday'], inplace=True)

In [ ]:
sns.scatterplot(
    data=train,
    x='distance',
    y='fare_amount',
    color=sns.color_palette("muted")[0],
)

plt.xlabel("Distancia (km)")
plt.ylabel("Tarifa (USD$)")
plt.title("Relación entre distancia y tarifa")
plt.show()

#### Variable *passenger_count*

In [ ]:
tarifa_promedio_por_cantidad_pasajeros = train.groupby("passenger_count")['fare_amount'].mean().reset_index(name="promedio")

plt.figure(figsize=(12, 6))
sns.barplot(
    data=tarifa_promedio_por_cantidad_pasajeros,
    x="passenger_count",
    y="promedio",
    hue="passenger_count",
    palette="muted",
    legend=False,
)

plt.title("Tarifa promedio por cantidad de pasajeros")
plt.xlabel("Cantidad de pasajeros")
plt.ylabel("Tarifa Promedio")

plt.tight_layout()
plt.show()

Si bien la cantidad exacta de pasajeros de un viaje es un dato desconocido antes de concretar el viaje, si sabemos que los viajes con 0 pasajeros son envíos mediante "Uber Flash", y los viajes de 5 o 6 pasajeros necesitan vehículos aprobados y se solicitan bajo el nombre "Uber XL". Est información si la tenemos disponible al momento de solicitar el viaje ya que forma parte de los parametros que configuran al mismo. 


A priori se observa que el promedio de tarifa de cadeteria mediante "Uber Flash" es inferior a cualquier viaje con pasajeros.

In [ ]:
train['type_of_service'] = np.where(train['passenger_count'] == 0, 'flash',
                                    np.where(train['passenger_count'] > 5, 'xl', 'x'))

test['type_of_service'] = np.where(test['passenger_count'] == 0, 'flash',
                                    np.where(test['passenger_count'] > 5, 'xl', 'x'))

tarifa_promedio_por_cantidad_pasajeros = train.groupby("type_of_service")['fare_amount'].mean().reset_index(name="promedio")

plt.figure(figsize=(12, 6))
sns.barplot(
    data=tarifa_promedio_por_cantidad_pasajeros,
    x="type_of_service",
    y="promedio",
    hue="type_of_service",
    palette="muted",
    legend=False,
)

plt.title("Tarifa promedio por cantidad de pasajeros")
plt.xlabel("Cantidad de pasajeros")
plt.ylabel("Tarifa Promedio")

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))

sns.boxplot(
    data=train[train['fare_amount'].quantile(0.99) >= train['fare_amount']], # Omite outliers para mejor visualización
    x='fare_amount',
    hue='type_of_service',
    palette='muted',
)

plt.title('Distribución de valor de tarifa por tipo de servicio')
plt.xlabel('Tarifa (USD)')
plt.legend(title='Tipo de servicio')
plt.tight_layout()
plt.show()

Se observan diferencias claras entre las tarifas de los distintos servicios, consistentemente las tarifas de servicio "Uber Flash" son inferiores a las de "Uber X", que son inferiores a las de "Uber XL". Vamos a considerar esta información para el modelo mediante el uso de variables dummys.

In [ ]:
# Genera dummys para tipo de servicio
train['is_xl'] = np.where(train['type_of_service'] == 'xl', 1, 0)
train['is_flash'] = np.where(train['type_of_service'] == 'flash', 1, 0)

test['is_xl'] = np.where(test['type_of_service'] == 'xl', 1, 0)
test['is_flash'] = np.where(test['type_of_service'] == 'flash', 1, 0)

## PreTrain

In [ ]:
train.columns

In [ ]:
# Elimina columnas que no se van a utilizar en el modelo
train.drop(columns=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'type_of_service', 'date', 'hour', 'day', 'month'], inplace=True)
test.drop(columns=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'type_of_service', 'date', 'hour', 'day', 'month'], inplace=True)